In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

df = pd.read_csv('credit_card_churn.csv')
df = df.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2', 'CLIENTNUM'], axis=1)
cols = df.columns
num_data = list(df._get_numeric_data().columns)
cat_data = list(set(cols) - set(num_data))
num_data.remove('Dependent_count')
num_data.remove('Total_Relationship_Count')
num_data.remove('Months_Inactive_12_mon')
num_data.remove('Contacts_Count_12_mon')
cat_data.append('Dependent_count')
cat_data.append('Total_Relationship_Count')
cat_data.append('Months_Inactive_12_mon')
cat_data.append('Contacts_Count_12_mon')
num_data_plot = num_data
cat_data_plot = cat_data

df = df.loc[df['Card_Category'] != 'Platinum']
df = df.loc[df['Total_Ct_Chng_Q4_Q1'] <= 1.6]
df = df.loc[df['Months_Inactive_12_mon'] >= 1]
df = df.loc[df['Months_Inactive_12_mon'] < 6]
df = df.loc[df['Customer_Age'] <= 66]
df = df.loc[df['Total_Amt_Chng_Q4_Q1'] <= 1.5]
df['Credit_Limit'] = np.log10(df['Credit_Limit'])
fitted_data, fitted_lambda = stats.boxcox(df['Avg_Open_To_Buy'])
df['Avg_Open_To_Buy'] = fitted_data
df['Total_Trans_Amt'] = np.log10(df['Total_Trans_Amt'])
df['Avg_Utilization_Ratio'] = (df['Avg_Utilization_Ratio'])**(1/2)
y = df['Attrition_Flag']
x = df.drop(['Attrition_Flag'], axis=1)
x = pd.get_dummies(x)
y = y.replace(['Existing Customer', 'Attrited Customer'], [1, 0])

In [ ]:
from sklearn.decomposition import PCA
for i in range(36):
    pca = PCA(n_components=(i+1))
    x_pca = pca.fit_transform(x)
    print(i+1)
    print(np.max(pca.explained_variance_ratio_))

In [2]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

x = x.values
y = y.values
scaler = MinMaxScaler()
x = scaler.fit_transform(x)
kf = KFold()
kf.get_n_splits(x)
for train_index, test_index in kf.split(x):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [1958 1959 1960 ... 9787 9788 9789] TEST: [   0    1    2 ... 1955 1956 1957]
TRAIN: [   0    1    2 ... 9787 9788 9789] TEST: [1958 1959 1960 ... 3913 3914 3915]
TRAIN: [   0    1    2 ... 9787 9788 9789] TEST: [3916 3917 3918 ... 5871 5872 5873]
TRAIN: [   0    1    2 ... 9787 9788 9789] TEST: [5874 5875 5876 ... 7829 7830 7831]
TRAIN: [   0    1    2 ... 7829 7830 7831] TEST: [7832 7833 7834 ... 9787 9788 9789]


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


parameters = {'C':[0.01, 0.1, 1, 10, 100],
                'solver':('newton-cg', 'liblinear', 'sag', 'saga')}
logisReg = LogisticRegression()
clf = GridSearchCV(logisReg, parameters, scoring='f1')
clf.fit(x_train, y_train)

print(clf.best_params_)
y_pred = clf.predict(x_test)
print(classification_report(y_test, y_pred))